In [1]:
%cd ..

/users/5/dever120/PyGRANSO


# Robust PCA

Solve the robust PCA problem taken from Xinyang Yi, et al. ["Fast algorithms for robust PCA via gradient
descent."](https://papers.nips.cc/paper/2016/hash/b5f1e8fb36cd7fbeb7988e8639ac79e9-Abstract.html) Advances in neural information processing systems. 2016.


## Problem Description

$$\min_{M,S}||M||_{\text{nuc}}+\lambda||S||_1,$$
$$\text{s.t. }Y=M+S,$$
where $M,S\in R^{d_1,d_2}$ are matrix form optimization variables, $Y\in R^{d_1,d_2}$ is a given matrix, and $||\cdot||_{\text{nuc}}$ denotes the nuclear norm.

## Modules Importing
Import all necessary modules and add PyGRANSO src folder to system path.

In [2]:
import time
import torch
from pygranso.pygranso import pygranso
from pygranso.pygransoStruct import pygransoStruct

## Data Initialization 
Specify torch device, and generate data.

Use GPU for this problem. If no cuda device available, please set *device = torch.device('cpu')*

In [3]:
device = torch.device('cpu')
d1 = 3
d2 = 4
torch.manual_seed(1)
eta = .05

# All the user-provided data (vector/matrix/tensor) must be in torch tensor format. 
# As PyTorch tensor is single precision by default, one must explicitly set `dtype=torch.double`.
# Also, please make sure the device of provided torch tensor is the same as opts.torch_device.
Y = torch.randn(d1, d2).to(device=device, dtype=torch.double)

## Function Set-Up

Encode the optimization variables, and objective and constraint functions.

Note: please strictly follow the format of comb_fn, which will be used in the PyGRANSO main algortihm.

In [4]:
# variables and corresponding dimensions.
var_in = {"M": [d1, d2],"S": [d1, d2]}


def user_fn(X_struct,Y):
    M = X_struct.M
    S = X_struct.S
    
    # objective function
    f = torch.norm(M, p='nuc') + eta * torch.norm(S, p=1)

    # inequality constraint, matrix form
    ci = None
    
    # equality constraint 
    ce = pygransoStruct()
    ce.c1 = M + S - Y

    return [f,ci,ce]

comb_fn = lambda X_struct : user_fn(X_struct, Y)

## User Options
Specify user-defined options for PyGRANSO

In [5]:
opts = pygransoStruct()
opts.torch_device = device
opts.print_frequency = 10
opts.x0 = .2 * torch.ones((2 * d1 * d2, 1)).to(device=device, dtype=torch.double)
opts.opt_tol = 1e-6

## Main Algorithm

In [6]:
start = time.time()
cpu_soln = pygranso(var_spec=var_in, combined_fn=comb_fn, user_opts=opts)
end = time.time()
print("Total Wall Time: {}s".format(end - start))



╔═════ QP SOLVER NOTICE ════════════════════════════════════════════════════════════════════════╗
║  PyGRANSO requires a quadratic program (QP) solver that has a quadprog-compatible interface,  ║
║  the default is osqp. Users may provide their own wrapper for the QP solver.                  ║
║  To disable this notice, set opts.quadprog_info_msg = False                                   ║
╚═══════════════════════════════════════════════════════════════════════════════════════════════╝
═════════════════════════════════════════════════════════════════════════════════════════════════════════════════╗
PyGRANSO: A PyTorch-enabled port of GRANSO with auto-differentiation                                             ║ 
Version 1.2.0                                                                                                    ║ 
Licensed under the AGPLv3, Copyright (C) 2021-2022 Tim Mitchell and Buyun Liang                                  ║ 
═════════════════════════════════════════════

## CUDA Computations

In [7]:
device = torch.device('cuda')
d1 = 3
d2 = 4
torch.manual_seed(1)
eta = .05

# All the user-provided data (vector/matrix/tensor) must be in torch tensor format. 
# As PyTorch tensor is single precision by default, one must explicitly set `dtype=torch.double`.
# Also, please make sure the device of provided torch tensor is the same as opts.torch_device.
Y = torch.randn(d1, d2).to(device=device, dtype=torch.double)

In [8]:
# variables and corresponding dimensions.
var_in = {"M": [d1, d2],"S": [d1, d2]}


def user_fn(X_struct,Y):
    M = X_struct.M
    S = X_struct.S
    
    # objective function
    f = torch.norm(M, p='nuc') + eta * torch.norm(S, p=1)

    # inequality constraint, matrix form
    ci = None
    
    # equality constraint 
    ce = pygransoStruct()
    ce.c1 = M + S - Y

    return [f, ci, ce]

comb_fn = lambda X_struct : user_fn(X_struct, Y)

In [9]:
opts = pygransoStruct()
opts.torch_device = device
opts.print_frequency = 10
opts.x0 = .2 * torch.ones((2 * d1 * d2, 1)).to(device=device, dtype=torch.double)
opts.opt_tol = 1e-6

In [10]:
start = time.time()
cuda_soln = pygranso(var_spec=var_in, combined_fn=comb_fn, user_opts=opts)
end = time.time()
print("Total Wall Time: {}s".format(end - start))



╔═════ QP SOLVER NOTICE ════════════════════════════════════════════════════════════════════════╗
║  PyGRANSO requires a quadratic program (QP) solver that has a quadprog-compatible interface,  ║
║  the default is osqp. Users may provide their own wrapper for the QP solver.                  ║
║  To disable this notice, set opts.quadprog_info_msg = False                                   ║
╚═══════════════════════════════════════════════════════════════════════════════════════════════╝
═════════════════════════════════════════════════════════════════════════════════════════════════════════════════╗
PyGRANSO: A PyTorch-enabled port of GRANSO with auto-differentiation                                             ║ 
Version 1.2.0                                                                                                    ║ 
Licensed under the AGPLv3, Copyright (C) 2021-2022 Tim Mitchell and Buyun Liang                                  ║ 
═════════════════════════════════════════════

## Testing

In [14]:
torch.testing.assert_close(
    cpu_soln.final.x,
    cuda_soln.final.x.cpu(),
    atol=1e-3,
    rtol=1e-3,
)